In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('/projects/cft_vlm/finetune')
import transformers
from transformers import AutoProcessor
import datasets
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", use_fast=True)


You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [3]:
from transformers import Qwen2_5_VLForConditionalGeneration

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
  "Qwen/Qwen2.5-VL-3B-Instruct",
  torch_dtype="auto",
  device_map="auto", 
  attn_implementation="flash_attention_2")

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
import torch

from qwenvl.argument import ModelArguments, DataArguments, ProcessingArguments
torch.set_num_threads(1)
parser = transformers.HfArgumentParser((
    ModelArguments,
    DataArguments,
    ProcessingArguments,
))

from qwenvl.data import logger
from qwenvl.train import set_processor, make_data_module

model_args, data_args, proc_args = ModelArguments(), DataArguments(), ProcessingArguments()
logger.info("Counting tokens, not training.")
processor = transformers.AutoProcessor.from_pretrained(
    model_args.model_name_or_path,
    use_fast=True,
)
data_args.dataset_use = 'vqa-rad'
processor = set_processor(processor, proc_args, data_args)

2025-06-10 17:35:43,487 - qwenvl.data - INFO - Counting tokens, not training.


In [8]:
data_module = make_data_module(
    processor=processor, data_args=data_args, proc_args=proc_args, for_training=False
)

2025-06-10 17:36:01,379 - qwenvl.data - INFO - Loading dataset flaviagiammarino/vqa-rad from /projects/cft_vlm/datasets/vqa_rad/data/dataset


In [26]:
from pathlib import Path
import torch
from qwenvl.new.predict import get_trainer_args, get_generated_text, save_result
from qwenvl.new.train import rank0_make_data_module
from transformers import Seq2SeqTrainer

data_args, proc_args, eval_args = DataArguments(), ProcessingArguments(), EvalArguments()

data_args.dataset_use = 'vqa-rad'
data_args.split = 'test'

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    eval_args.model_name_or_path,
    torch_dtype=torch.float16,
    attn_implementation="flash_attention_2"
)
processor = AutoProcessor.from_pretrained(eval_args.model_name_or_path)
processor = set_processor(processor, proc_args, data_args)
data_module = rank0_make_data_module(
    processor=processor,
    data_args=data_args,
    proc_args=proc_args,
    for_training=False
)
trainer_args = get_trainer_args(eval_args)

trainer = Seq2SeqTrainer(
    model=model,
    processing_class=processor,
    args=trainer_args,
    **data_module
)
output = trainer.predict(trainer.eval_dataset)
generated_text = get_generated_text(output, processor.tokenizer)
if eval_args.output_dir is not None:
  output_dir = Path(eval_args.output_dir)
else:
  output_dir = Path(eval_args.model_name_or_path)
save_result(data_module['eval_dataset'], generated_text, output_dir)
    

ModuleNotFoundError: No module named 'mpi4py'

In [ ]:
from qwenvl.data import OpenpmcDataset


ds = OpenpmcDataset(
    processor=processor,
    data_args=data_args,
    proc_args=proc_args,
    split='train',
)
